In [2]:
import sys
import json
import glob

sys.path.append('datasets')
sys.path.append('models')

# this is the default configuration
default = {"batch_size": 250,
          "chunks": False,
          "embeddings_model": "glo50",
          "epochs": 1000,
          "hidden_layers": [32],
          "input_labels": ["ID", "FORM", "MARKER", "GPOS", "FORM_CTX_P-1", "FORM_CTX_P+0", "FORM_CTX_P+1"],
          "lr": 0.005,
          "ru": "BasicLSTM",
          "target_labels": ["T"],
          "kfold": False,
          "version": "1.0"}